In [ ]:
!pip install -q transformers kaggle jiwer
import os
from pathlib import Path
import pandas as pd
from transformers import pipeline
from google.colab import files
from jiwer import wer, compute_measures



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.1 MB/s eta 0:00:00


In [ ]:
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d thinh127/vivos-vietnamese -p /content
!unzip -q /content/vivos-vietnamese.zip -d /content/vivos-vietnamese



Dataset URL: https://www.kaggle.com/datasets/thinh127/vivos-vietnamese
License(s): unknown
 98% 1.34G/1.37G [00:10<00:00, 250MB/s]
100% 1.37G/1.37G [00:10<00:00, 134MB/s]


In [ ]:
BASE   = Path("/content/vivos-vietnamese")
TEST   = BASE / "vivos" / "test"
WAVES  = TEST / "waves"
PROMPT = TEST / "prompts.txt"
GENDER = TEST / "genders.txt"



In [ ]:
rows = []
with open(PROMPT, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        utt, text = line.split(" ", 1)
        rows.append({"utterance_id": utt, "prompt": text})
prompts = pd.DataFrame(rows)

In [ ]:
gdf = pd.read_csv(GENDER, sep=" ", header=None, names=["speaker_id","gender"])
gender_map = dict(zip(gdf.speaker_id, gdf.gender))



In [ ]:
asr = pipeline(
    "automatic-speech-recognition",
    model="vinai/PhoWhisper-medium",
    device=0
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
results = []
for _, row in prompts.iterrows():
    utt     = row["utterance_id"]
    speaker = utt.split("_")[0]
    wav     = WAVES / speaker / f"{utt}.wav"

    if not wav.exists():
        print(f"⚠️  Missing: {wav}")
        continue

    pred = asr(str(wav))["text"]
    results.append({
        "utterance_id": utt,
        "prompt":       row["prompt"],
        "gender":       gender_map.get(speaker, "unknown"),
        "prediction":   pred
    })

df = pd.DataFrame(results)
display(df.head())



/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warni

,utterance_id,prompt,gender,prediction
0,VIVOSDEV02_R106,TRỞ NÊN THỤ ĐỘNG,m,trở nên thụ động.
1,VIVOSDEV02_R122,CŨNG KHIẾN CHO HỌ DÈ DẶT,m,cũng khiến cho họ dè dặt.
2,VIVOSDEV02_R130,CHỊ GẶN HỎI ANH THỀ SỐNG THỀ CHẾT LÀ KHÔNG CÓ,m,chị cảnh hỏi anh thề sống thề chết là không có.
3,VIVOSDEV02_R132,ĐIỆN THOẠI RENG NHƯNG TA CŨNG PHẢI NHẤC MÁY ĐÚ...,m,tiện thoại ren nhưng ta cũng phải nhấc máy đún...
4,VIVOSDEV02_R135,CŨNG THUỘC LOẠI DÀY ĐẶC NHẤT TRONG HỆ MẶT TRỜI,m,cũng thuộc loại giày đặc nhất trong hệ mặt trời.


In [ ]:
df["prompt"]     = df["prompt"].str.lower().str.strip()
df["prediction"] = df["prediction"].str.lower().str.strip()



In [ ]:
refs = df["prompt"].tolist()
hyps = df["prediction"].tolist()
print(f"Global WER: {wer(refs, hyps):.2%}")
df["wer"] = df.apply(lambda r: wer(r.prompt, r.prediction), axis=1)
print(f"Mean utterance-level WER: {df.wer.mean():.2%}")
meas = compute_measures(refs, hyps)
print("Substitutions:", meas["substitutions"])
print("Deletions:    ", meas["deletions"])
print("Insertions:   ", meas["insertions"])



Global WER: 14.31%
Mean utterance-level WER: 16.29%
Substitutions: 1070
Deletions:     22
Insertions:    13


In [ ]:
with open("transcripts.txt", "w", encoding="utf-8") as f:
    for pred in df["prediction"]:
        f.write(pred + "\n")

print("✅ transcripts.txt ready—one lowercase line per file.")

✅ transcripts.txt ready—one lowercase line per file.
